# 3D SE-FFN: Training
A script for repeatedly training the SE-FFN.

Due to the model's run time complexity, the 10^4 epoch runs each take ~3-5mins (Colab external runtime). The file's total runtime was 25-55mins with 10 trials (early-stopping dependent).

In [1]:
# Reproducibility Seeding
import os, random, numpy as np
seed = 42
np.random.seed(seed); random.seed(seed)

# Preamble


Import Packages.

In [2]:
# Maths/data
import numpy as np
from numpy import array
import math
import pandas as pd
import time

# California Housing Dataset import and preprocessing
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Copying/nesting
import copy
from copy import deepcopy
from collections import defaultdict

# Static Plotting
import matplotlib.pyplot as plt
plt.rcParams.update({
    # "font.family": "serif",  # Use a serif font (e.g., Times)
    # "font.serif": "Computer Modern Roman", # Or "Times New Roman"
    "font.size": 10,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 10,
    "figure.figsize": (6, 4), # Adjust figure size
    "axes.grid": True,
    "grid.alpha": 0.5, # Make grid more subtle
    "lines.linewidth": 1.5 # Make lines slightly thicker
})
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

# Save and export data
import gzip, pickle, types, pathlib
from google.colab import files

# Data Pre-processing

Import and clean data. Normalise data to mean 0 and std dev 1.

In [3]:
# Download the dataset
data = fetch_california_housing()
X_all = pd.DataFrame(data.data, columns=data.feature_names)
y_df = pd.Series(data.target, name='MedHouseVal')
X_df = X_all[['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Latitude', 'Longitude', 'Population', 'AveBedrms']]

# DF -> numpy arrays
X = X_df.to_numpy()
y = y_df.to_numpy()

# Prartition into training (70%), validation (15%) and test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Reshape outputs (N_obs, N_features)
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)
y_test = y_test.reshape(-1,1)

# Scale inputs
scaler_X = StandardScaler()
X_tr = scaler_X.fit_transform(X_train)
X_vl = scaler_X.transform(X_val)
X_tt = scaler_X.transform(X_test)

# Scale outputs
scaler_Y = StandardScaler()
y_tr = scaler_Y.fit_transform(y_train)
y_vl = scaler_Y.transform(y_val)
y_tt = scaler_Y.transform(y_test)

# Network Initialisation

Distribute I-O nodes on parallel $Z_2Z_3$ planes: $Z^{(0)}, Z^{(L-1)}$

In [4]:
def points_on_square(N, side_length=1, x=0):
    n = int(math.ceil(math.sqrt(N)))
    step = side_length / (n - 1) if n > 1 else side_length / 2

    points = []
    for i in range(n):
        for j in range(n):
            y = i * step
            z = j * step
            points.append((x, y, z))
            if len(points) == N:
                return np.array(points)

    # Randomly remove excess points
    if len(points) > N:
        points = np.array(points)
        np.random.shuffle(points)
        return points[:N]

    return np.array(points)

Layer index sorter.

In [5]:
def l_sort(norm_dict):
    l_unique = set()
    for (a, b) in norm_dict.keys():
        l_unique.add(a)
        l_unique.add(b)
    return l_unique

Construct network graph dict:
$$
\mathcal{G} = \bigl\{\,
0 \;\mapsto\; \{Z^{(0)}, W^{(0)}, b^{(0)}\}\;,\dots,\;L-1 \;\mapsto\; \{Z^{(L-1)}, W^{(L-1)}, b^{(L-1)}\}
\;,\bigr\}
$$

In [6]:
def build_net(l_list, D=3):
    G = {}
    for l, N_l in enumerate(l_list):
        Gl = {}

        if l == 0:
            # 1) Fixed input grid, no parameters
            N_0 = l_list[0]
            Gl["Z"] = points_on_square(N_0, side_length=1, x=0)
            Gl["W"] = None
            Gl["b"] = None

        elif l in range(1, len(l_list)-1):
            # 2) Weights from previous to current
            Gl["Z"] = np.random.uniform(0.1, 0.9, size=(l_list[l],D))
            Gl["W"] = np.ones((N_l, l_list[l-1]))
            Gl["b"] = np.ones(N_l)

        elif l == len(l_list)-1:

            # 3) Fixed output nodes, weights from previous
            N_L = l_list[l]
            Gl["Z"] = np.array([[1, 0.5, 0.5]]) # distribute_points_on_square(N_L, side_length=1, x=1)
            Gl["W"] = np.ones((1, l_list[l-1]))
            Gl["b"] = np.zeros(N_l)

        G[l] = Gl

    return G
# Network performs poorly when initialising all nodes symmetrically.

# Parameter Conditions

## Norms

Build node norms dict:
$$
\mathcal{Q} = \bigl\{\,
(1,\; 0) \;\mapsto\; ||Z^{(1)}-Z^{(0)}||\;,\dots,\; (L-1,\; L-2) \;\mapsto\; ||Z^{(L-1)}-Z^{(L-2)}||
\,\bigr\}
$$

In [7]:
def norm_dict(G):
    L = max(G.keys())
    Q = {}

    # Norm dimension-wise differences
    for l in range(L):
        Z, Zf = G[l]['Z'], G[l+1]['Z']                 # Zb
        Δ = Zf[:, None, :] - Z[None, :, :]             # (n_{l+1}, n_l, D)
        Q[(l+1, l)] = np.linalg.norm(Δ, axis=2) + ϵ    # (n_{l+1}, n_l)

    return Q

## Weights
Weight condition and derivative:

$$
w_{n_{l+1, l}, n_{l}} = w_{max} \cdot tanh(s_w \cdot (q_{n_{l+1}, n_l}-q_0))
$$

In [8]:
def compute_w(q, s_w, w_max, q0):
    return  w_max * -np.tanh(s_w * (q-q0))

def compute_dw(q, s_w, w_max, q0):
    # sech²(x) = 1/cosh²(x)
    sech_sq = 1.0 / np.cosh(s_w * (q - q0))**2
    return -w_max * s_w * sech_sq

Build weight dict:

$$
\mathcal{W} = \bigl\{\,
(1,\; 0) \;\mapsto\; W^{(1,\; 0)}\;,\dots,\; (L-1,\; L-2) \;\mapsto\; W^{(L-1,\; L-2)}
\,\bigr\}
$$

<br>where:<br>

$$
W^{(l+1,\; l)} = \begin{bmatrix} w^{(l+1)}_{1, 1} \;\dots\; w^{(l+1)}_{1, n_l}\\\vdots \;\ddots\; \vdots\\w^{(l+1)}_{n_{l+1}, 1} \;\dots\; w^{(l+1)}_{n_{l+1}, n_l}\end{bmatrix}
$$

In [9]:
def weight_dict(Q):
    ls = l_sort(Q)
    W = {}
    for l in sorted(ls):
      if (l+1, l) in Q:
        W[(l+1, l)] = compute_w(Q[(l+1, l)], s_w, w_max, q0)
    return W

## Combined Condition (Bias Removed)
Apply parameter conditions to net.

In [10]:
def update_params(G):
    Q = norm_dict(G)
    # B = bias_dict(Q)
    W = weight_dict(Q)

    # G <- new params
    ls = l_sort(Q)
    for l in range(len(ls) - 1):
        N_f = G[l+1]["Z"].shape[0]
        Gl = {}
        Gl["Z"] = G[l+1]["Z"]
        Gl["W"] = W[(l+1, l)]
        Gl["b"] = np.zeros((N_f,))

        # Move to next layer
        G[l+1] = Gl

    return G

# Forward Propagation

Define forward propagation.

In [11]:
def forward(X, G):
    cache = {}
    A_prev = X
    cache["A0"] = X  # store input as A0

    for l in sorted(i for i in G if i != 0):
        Wl = G[l]["W"]                        # (N_(l-1), N_l)
        bl = G[l]["b"]                        # (1, N_l)
        Al = np.dot(A_prev, Wl.T) + bl        # (N_l, N)
        cache[f"A{l}"] = Al
        A_prev = Al

    AL = A_prev
    return AL, cache

# Backpropagation

## Loss and Regulariser
Regularised loss:
$\mathcal{L}_{reg} = \frac{1}{2} \Sigma_{l} Σ_{n_{l+1},n_l} [s_{reg} · (q^{(l+1)}_{n_{l+1}, n_l} - q_0)]^2$


In [12]:
def loss_reg(G):
    l_reg = 0.0
    Q = norm_dict(G)
    for q in Q.values():
        l_reg += np.sum((s_reg * (q - q0)) ** 2)
    return l_reg

def loss_pred(Y, AL):
    NL, N = Y.shape
    return (1 / (2 * NL * N)) * np.sum((Y - AL)**2)

def loss(Y, AL, G):
    return loss_pred(Y, AL) + loss_reg(G)

## Parameter Step
Define the conventional backpropagation algorithm.

$$\frac{d\mathcal{L}}{d\theta} = \bigl\{\,
(1,\; 0) \;\mapsto\; \{\frac{d\mathcal{L}}{dW^{(1,\; 0)}}, \frac{d\mathcal{L}}{db^{(1)}}\}\;,\dots,\; (L-1,\; L-2) \;\mapsto\; \{\frac{d\mathcal{L}}{dW^{(L-1,\; L-2)}}, \frac{d\mathcal{L}}{db^{(L-1)}}\}
\,\bigr\}
$$

In [13]:
def back_θ(Y, cache, G):
    dθ = {}
    L = len(G)-1
    N = Y.shape[0]
    NL = Y.shape[1]
    AL = cache[f"A{L}"]

    # dLdy_pred
    dAL = ( 1 / (NL * N) ) * (AL - Y)

    # Backpropagate layer-wise
    dA = dAL
    for l in reversed(range(1, L + 1)):
        Al = cache[f"A{l}"]
        Wl = G[l]["W"]

        # Compute gradients
        dW = np.dot(dA.T, cache[f"A{l-1}"])
        db = np.sum(dA, axis=0)
        dA = np.dot(dA, Wl)

        # Store gradients
        dθ[f"dW{l}"] = dW
        dθ[f"db{l}"] = db

    return dθ

## Coordinate Step

### Weights

Build weight-coordinate derivatives $\frac{\partial W}{\partial Z^{(l+1)}}$ and $\frac{\partial W}{\partial Z^{(l)}}$:

<br>

Forward:
$$
\underbrace{\frac{\partial W^{(l+1)}}{\partial Z^{(l+1)}}}_{(N_{l+1},\; N_l,\; D)}\ = \begin{bmatrix}\frac{\partial w_{1, 1}}{\partial z_{1,d}}\; \dots \; \frac{\partial w_{1,n_l}}{\partial z_{1,d}} \\\vdots \; \ddots \; \vdots\\ \frac{\partial w_{n_{l+1}, 1}}{\partial z_{n_{l+1},d}}\; \dots \; \frac{\partial w_{n_{l+1}, n_l}}{\partial z_{n_{l+1}, d}}\end{bmatrix}^{d=1} \dots \quad \begin{bmatrix}\frac{\partial w_{1, 1}}{\partial z_{1,d}}\; \dots \; \frac{\partial w_{1,n_l}}{\partial z_{1,d}} \\\vdots \; \ddots \; \vdots\\ \frac{\partial w_{n_{l+1}, 1}}{\partial z_{n_{l+1}l,d}}\; \dots \; \frac{\partial w_{n_{l+1}, n_l}}{\partial z_{n_{l+1}, d}}\end{bmatrix}^{d=D}$$


<br>

Backward:
$$
\underbrace{\frac{\partial W^{(l+1)}}{\partial Z^{(l)}}}_{(N_{l+1},\; N_l,\; D)}\ = \begin{bmatrix}\frac{\partial w_{1, 1}}{\partial z_{1,d}}\; \dots \; \frac{\partial w_{1,n_l}}{\partial z_{n_l,d}} \\\vdots \; \ddots \; \vdots\\ \frac{\partial w_{n_{l+1}, 1}}{\partial z_{1,d}}\; \dots \; \frac{\partial w_{n_{l+1}, n_l}}{\partial z_{n_l, d}}\end{bmatrix}^{d=1} \dots \quad \begin{bmatrix}\frac{\partial w_{1, 1}}{\partial z_{1,d}}\; \dots \; \frac{\partial w_{1,n_l}}{\partial z_{n_l,d}} \\\vdots \; \ddots \; \vdots\\ \frac{\partial w_{n_{l+1}, 1}}{\partial z_{1,d}}\; \dots \; \frac{\partial w_{n_{l+1}, n_l}}{\partial z_{n_l, d}}\end{bmatrix}^{d=D}$$

In [14]:
def get_dWdZ(Zf, Z, ϵ):
    Δ = Zf[:, np.newaxis, :] - Z[np.newaxis,:, :]    # (N_{l+1}, N_l, D)
    Ql = np.sqrt(np.sum(Δ**2, axis=-1) + ϵ)          # (N_{l+1}, N_l)

    dWdQl = compute_dw(Ql, s_w, w_max, q0)

    # Compute derivatives w.r.t. coordinates
    s = (dWdQl / Ql)[..., np.newaxis]       # (N_{l+1}, N_l, 1)
    dWdZ = -s * Δ                           # (N_{l+1}, N_l, D)
    dWdZf = -dWdZ

    return dWdZ, dWdZf

Compile loss-coordinates derivative dict. via weights $\frac{\partial \mathcal{L}}{\partial Z}$.

<br>

$$
\small{
\left(\frac{\partial \mathcal{L}}{\partial Z^{(l)}}\right)_{n_l, d} = \sum_{n_{l+1}}\left[\left(\frac{\partial \mathcal{L}}{\partial W^{(l+1)}}\right)_{n_{l+1}, n_l} \cdot \left(\frac{\partial W^{(l+1)}}{\partial Z^{(l)}}\right)_{n_{l+1}, n_l, d}\right]
+
\sum_{n_{l-1}}\left[\left(\frac{\partial \mathcal{L}}{\partial W^{(l)}}\right)_{n_l, n_{l-1}} \cdot \left(\frac{\partial W^{(l)}}{\partial Z^{(l)}}\right)_{n_{l+1}, n_l, d}\right]
}
$$

<br>

$$
\frac{\partial \mathcal{L}}{\partial Z^{(l)}} = \begin{bmatrix}\frac{\partial \mathcal{L}}{\partial z_{1, 1}} & \dots & \frac{\partial \mathcal{L}}{\partial z_{1, D}}\\ \vdots & \ddots & \vdots\\ \frac{\partial \mathcal{L}}{\partial z_{N_l, 1}} & \dots & \frac{\partial \mathcal{L}}{\partial z_{N_l, D}}\end{bmatrix}
$$


In [15]:
def back_W(dθ, G):
    dWdZ = {}
    dLdZ_W = {}
    L = len(G) - 1

    for l in range(1, L):
        Z = G[l]["Z"]           # (N_l, D)
        Zf = G[l+1]["Z"]        # (N_{l+1}, D)
        Zb = G[l-1]["Z"]        # (N_{l-1}, D)

        # Forward differences: current → next
        dWdZf, _ = get_dWdZ(Zf, Z, ϵ)

        # Backward differences: current → previous
        _, dWdZb = get_dWdZ(Z, Zb, ϵ)

        # Store dWdZ
        dWdZ[l] = {
            'forward': dWdZf,
            'backward': dWdZb
        }

        # Chain dLdW and dWdZ (forward)
        dLdWf = dθ[f"dW{l+1}"]
        dLdZf = np.einsum('ab,abd->bd', dLdWf, dWdZ[l]['forward'])

        # Chain dLdW and dWdZ (backward)
        dLdWb = dθ[f"dW{l}"]
        dLdZb = np.einsum('bc,bcd->bd', dLdWb, dWdZ[l]['backward'])

        dLdZ_W[l] = dLdZf + dLdZb

    return dLdZ_W

### Regulariser

Regularisation derivative:  $\frac{\partial L_{reg}}{\partial q_{n_{l+1},n_l}} = s_{reg}^2 \cdot (q_{n_{l+1},n_l}-q_0)$

In [16]:
# Compute a dict like norms containing dL_reg/dQ_{ji}
def get_dLdQ_r(Q):
    s_reg2      = s_reg * s_reg
    dLdQ  = {}
    for key, q in Q.items():
        dLdQ[key] = s_reg2 * (q - q0)    # (N_{l+1}, N_L)
    return dLdQ

Get loss-coordinate derivatives via regulariser.

In [17]:
def back_r(G, Q):

    # Count Layers, calculate total distance, intialise grad dicts
    L = max(G.keys())
    dLdZ_r = {l: np.zeros_like(G[l]['Z']) for l in range(L + 1)}
    dLdQ_r = get_dLdQ_r(Q)

    # Loop over layers, calculate loss-coord grads (via reg)
    for l in range(L):
        Z, Zf = G[l]['Z'], G[l+1]['Z']
        Ql    = Q[(l+1, l)]                       # (n_{l+1}, n_l)
        Δ    = Zf[:, None, :] - Z[None, :, :]     # (n_{l+1}, n_l, D)

        # chain: dL/dZ = dL/dQl · dQl/dZ
        grad = dLdQ_r[(l+1, l)][:, :, None] * Δ / Ql[:, :, None]
        dLdZ_r[l+1] += np.sum(grad, axis=1)  # l+1
        dLdZ_r[l]   -= np.sum(grad, axis=0)  # l

    return dLdZ_r

# Update

## Optimiser
AdaM for efficient solution exploration.

In [18]:
def adam(Z, dLdZ, AS, lr=1e-3, r1=0.9, r2=0.999):
    m, v, t = AS
    t += 1

    # 1st & 2nd moment updates
    m = r1 * m + (1 - r1) * dLdZ
    v = r2 * v + (1 - r2) * (dLdZ * dLdZ)

    # Bias correction
    m_hat = m / (1 - r1 ** t)
    v_hat = v / (1 - r2 ** t)

    # Coord update
    Z = Z - lr * m_hat / (np.sqrt(v_hat) + ϵ)

    return Z, (m, v, t)

## Gradient Descent
Define gradient descent loop.

In [19]:
from logging import addLevelName
def GD(G, eps, its, lr):
    # Nested dict (epoch -> iter -> l -> Z)
    hy = defaultdict(lambda: defaultdict(dict))
    hy_tr = []
    hy_vl = []

    # Track validation loss for early stopping
    l_pr_vlg = float('inf')
    ep_unimproved = 0
    Gg = None

    AS = {
    l: {
        "m": np.zeros_like(G[l]["Z"]),
        "v": np.zeros_like(G[l]["Z"]),
        "t": 0
    }
    for l in range(1, len(G)-1)
    }

    # Loop over epochs and iterations
    for ep in range(eps):
        for it in range(its):

            # 1-2) Forward pass, backprop
            y_pr, cache = forward(X_tr, G)
            dθ = back_θ(y_tr, cache, G)

            # 3) Coords backprop (W,b,L_reg)
            Q = norm_dict(G)
            dLdZ_W = back_W(dθ, G)
            dLdZ_r = back_r(G, Q)

            # 4) Sum coord grads. Update coords
            for l in range(1, len(G)-1):
                dLdZ = dLdZ_W.get(l, 0) + dLdZ_r.get(l, 0)

                # get AdaM states, apply update
                m, v, t   = (AS[l]["m"], AS[l]["v"], AS[l]["t"])
                G[l]["Z"], (m, v, t) = adam(G[l]["Z"], dLdZ, (m, v, t), lr=lr, r1=0.9, r2=0.999)
                AS[l].update({"m": m, "v": v, "t": t})

            # 5) Update Params
            G = update_params(G)

            # 6) Record layer coords.
            for l in range(0, len(G)):
              hy[ep][it][l] = G[l]["Z"].copy()

            # 7) Compute training error.
            l_tr = loss(y_tr, y_pr, G)
            l_pr_tr = loss_pred(y_tr, y_pr)
            hy_tr.append(l_tr)

            # 8) Validation
            y_vlpr, _ = forward(X_vl, G)
            l_pr_vl = loss_pred(y_vl, y_vlpr)
            l_vl = loss(y_vl, y_vlpr, G)
            hy_vl.append(l_vl)
            #print(f"Ep. {ep+1}, It. {it+1}")
            #print(f"Ep. {ep+1}, It. {it+1}: Loss_tr = {l_tr}, Loss_Vl = {l_vl}, Loss_pr_tr = {l_pr_tr},  Loss_pr_vl = {l_pr_vl}")

        # Validation stop check
        if l_pr_vl < l_pr_vlg:
                l_pr_vlg = l_pr_vl
                Gg = deepcopy(G)
                ep_unimproved = 0
        else:
                ep_unimproved += 1
        """
        Optional: early stopping
        if ep_unimproved >= 100:
            print("Early stopping condition met.")
            #break
        """
    """
    # Plot the loss
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(hy_tr)), hy_tr, label='Training Loss')
    plt.plot(range(len(hy_vl)), hy_vl, label='Validation Loss')
    plt.yscale('log')
    plt.xlabel('Updates')
    plt.ylabel('(Log) Loss')
    plt.title('Loss vs Updates')
    plt.legend()
    plt.grid(True)
    plt.show()
    """

    return Gg, hy

# Main

In [20]:
ϵ = 1e-8

# Weight hyper-parameters
q0 = np.sqrt(3)/4 #1/4 works but central clumping occurs
w_max = 2
w_sat =  0.99   # saturation
q_sat = np.sqrt(3)/4

# The weight function saturates 100*w_sat (%) at d_s (units) about d_0
s_w = np.arctanh(w_sat) / q_sat

# Net shape
N = X_tr.shape[0]
N_0 = X_tr.shape[1]
N_L = y_tr.shape[1]
l_list = [N_0, 32, 8, 2, N_L]

# Initialise net
G = build_net(l_list)
G = update_params(G)

# Run backpropagation
s_reg = 1 #0.75

basic_run = False
if basic_run==True:
  G, hy = GD(G=G, eps=5000, its=1, lr=0.04)
  y_pr = forward(X_tt, G)[0]
  l_pr = loss_pred(y_tt, y_pr)
  l_reg = loss_reg(G)
  print(f"Test Pred Loss: {l_pr}.\n\nTest Reg. Loss:{l_reg}")

# Experiment (SE-FFN)

Execute multiple GD runs with different parameter settings.

In [ ]:
import pickle, copy
import numpy as np
import time, pathlib


repeats = 10
# Define settings (S) for testing
S = [
    {"lr": 0.04, "eps": 100, "its": 1, "name": "lr0.04_eps100"},
    {"lr": 0.04, "eps": 1000, "its": 1, "name": "lr0.04_eps1000"},
    {"lr": 0.04, "eps": 10000, "its": 1, "name": "lr0.04_eps10000"}
]

best_overall = {
    "val_loss": np.inf,
    "setting": None,
    "run_idx": None,
    "G_best": None,
    "history": None
}


all_results = {}


# Set output spacing
row_fmt = (
    "  run {run_idx:>2d}/{repeats:<2d} | "
    "train {tr:>9.4f} | "
    "val {vl:>9.4f} | "
    "test {tt:>9.4f} | "
    "{rt:>7.2f}s"
)
summary_fmt = "  {label:<10} μ: {mean:>10.6f}, σ: {std:>10.6f}"


save_path = pathlib.Path("artifacts")
save_path.mkdir(exist_ok=True)

print("Starting multiple SE‑FFN runs…")
print("=" * 60)

for s_idx, s in enumerate(S):
    s_name = s["name"]
    eps = s["eps"]
    its = s["its"]
    lr = s["lr"]

    # Sub-title line
    print(f"\nSetting {s_idx+1}: {s['name']}  (lr={s['lr']}, eps={s['eps']}, its={s['its']})")
    print("-" * 40)

    s_results = {"G_best": [], "train_loss": [], "val_loss": [], "test_loss": []}

    for r in range(repeats):
        t0 = time.time()

        # Initialize network and run optimiser
        G_new = build_net(l_list)
        G_new = update_params(G_new)
        G_best, hy = GD(G_new, eps=s["eps"], its=s["its"], lr=s["lr"])

        # Calculate final losses with the best G
        y_tr_pred, _ = forward(X_tr, G_best)
        y_vl_pred, _ = forward(X_vl, G_best)
        y_tt_pred, _ = forward(X_tt, G_best)

        train_loss = loss_pred(y_tr, y_tr_pred)
        val_loss   = loss_pred(y_vl, y_vl_pred)
        test_loss  = loss_pred(y_tt, y_tt_pred)

        # Store results
        s_results["G_best"].append(G_best)
        s_results["train_loss"].append(train_loss)
        s_results["val_loss"].append(val_loss)
        s_results["test_loss"].append(test_loss)

        # Global best cache update
        if val_loss < best_overall["val_loss"]:
            best_overall.update(
                val_loss = val_loss,
                setting  = s["name"],
                run_idx  = r + 1,
                G_best   = copy.deepcopy(G_best),   # deep‑copy to freeze weights
                history  = copy.deepcopy(hy)        # loss trajectory
            )

            # persist immediately so interrupted run retains the artefact
            # with open(save_path / "best_snapshot.pkl", "wb") as f:
                # pickle.dump(best_overall, f)

            print(f"\n ***New optimal model obtained***  Val={val_loss:.6f}  "
                  f"[{s['name']} | run {r+1}] (checkpoint saved)\n")

        # Results output line
        print(row_fmt.format(run_idx=r+1, repeats=repeats,
                             tr=train_loss, vl=val_loss,
                             tt=test_loss, rt=time.time()-t0))

    all_results[s["name"]] = s_results

    # Print summary statistics for current setting
    train_losses = s_results["train_loss"]
    val_losses = s_results["val_loss"]
    test_losses = s_results["test_loss"]

    print(f"\nSummary for {s_name}:")
    print(summary_fmt.format(label="Train Loss", mean=np.mean(train_losses), std=np.std(train_losses)))
    print(summary_fmt.format(label="Val Loss",   mean=np.mean(val_losses),   std=np.std(val_losses)))
    print(summary_fmt.format(label="Test Loss",  mean=np.mean(test_losses),  std=np.std(test_losses)))

# Comprehensive Results Summary
print("\n" + "="*60)
print("ALL RUNS COMPLETED!")
print("="*60)

print("\nBEST MODEL OVERALL")
print("-"*40)
print(f"Setting : {best_overall['setting']}")
print(f"Run     : {best_overall['run_idx']}/{repeats}")
print(f"Val loss: {best_overall['val_loss']:.6f}")
print(f"Saved   : {save_path/'best_snapshot.pkl'}")

Starting multiple SE‑FFN runs…

Setting 1: lr0.04_eps100  (lr=0.04, eps=100, its=1)
----------------------------------------

 ***New optimal model obtained***  Val=19.815297  [lr0.04_eps100 | run 1] (checkpoint saved)

  run  1/10 | train   20.8566 | val   19.8153 | test   29.0266 |    6.83s

 ***New optimal model obtained***  Val=4.674290  [lr0.04_eps100 | run 2] (checkpoint saved)

  run  2/10 | train    5.5018 | val    4.6743 | test    5.1793 |    8.53s
  run  3/10 | train   10.0038 | val    7.9407 | test    9.4946 |    2.22s
  run  4/10 | train   13.2707 | val   11.4931 | test   14.8666 |    1.47s

 ***New optimal model obtained***  Val=1.082112  [lr0.04_eps100 | run 5] (checkpoint saved)

  run  5/10 | train    2.5722 | val    1.0821 | test    1.0225 |    1.45s
  run  6/10 | train   22.1990 | val   23.0274 | test   23.7609 |    1.47s
  run  7/10 | train   22.5658 | val   17.4745 | test   27.6440 |    2.48s

 ***New optimal model obtained***  Val=0.878040  [lr0.04_eps100 | run 8] 

Save/export the optimal graph and update history.

In [ ]:
# Remove callable entries from the history file
def strip_callables(obj):
    if callable(obj):
        return None
    if isinstance(obj, dict):
        return {k: strip_callables(v) for k, v in obj.items()
                if not callable(v)}
    if isinstance(obj, list):
        return [strip_callables(v) for v in obj if not callable(v)]
    if isinstance(obj, tuple):
        return tuple(strip_callables(v) for v in obj if not callable(v))
    return obj                     # leaf

safe_history = strip_callables(best_overall["history"])

# Build the bundle for saving
bundle = {
    "G_best":  best_overall["G_best"],
    "history": safe_history,
    "l_list":  l_list,
    "meta": {
        "created":      time.strftime("%Y-%m-%d %H:%M:%S"),
        "val_loss":     float(best_overall["val_loss"]),
        "numpy_version": np.__version__,
    },
}

# Dump to compressed pickle file
out_path = pathlib.Path("g_and_history.pkl.gz")
with gzip.open(out_path, "wb") as f:
    pickle.dump(bundle, f, protocol=5)

print(" Saved:", out_path.resolve().as_posix())

# Download
files.download(str(out_path))

# Results Dataframe

Print results dataframe for further analysis.

In [ ]:
# Flatten results for DataFrame
df_data = []
for setting_name, results in all_results.items():
    for run in range(repeats):
        df_data.append({
            "Setting": setting_name,
            "Run": run + 1,
            "Train_Loss": results["train_loss"][run],
            "Val_Loss": results["val_loss"][run],
            "Test_Loss": results["test_loss"][run]
        })

results_df = pd.DataFrame(df_data)
print("\nResults DataFrame:")
print(results_df)

# Summary statistics
print("\nSummary Statistics by Setting:")
print("=" * 60)
summary_stats = results_df.groupby('Setting')[['Train_Loss', 'Val_Loss', 'Test_Loss']].agg(['mean', 'std', 'min', 'max'])
print(summary_stats)